# MOUNT GOOGLE DRIVE & PATH

In [2]:
from google.colab import drive

drive.mount('/content/drive', force_remount=True)

PROJECT_ROOT = "/content/drive/MyDrive/bigdata_final_project"
RAW_PATH       = f"{PROJECT_ROOT}/raw"
WAREHOUSE_PATH = f"{PROJECT_ROOT}/warehouse"
DB_PATH        = f"{WAREHOUSE_PATH}/bigdata_warehouse.db"

Mounted at /content/drive


# IMPORT LIBRARY

In [3]:
import pandas as pd
import sqlite3
from datetime import datetime


# LOAD CSV TRANSJAKARTA (RAW)

In [4]:
taxi_file = f"{RAW_PATH}/raw_taxi_data.csv"

df_taxi_raw = pd.read_csv(taxi_file)


In [5]:


df_taxi_raw.head()
df_taxi_raw.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150000 entries, 0 to 149999
Data columns (total 7 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   pickup_datetime        150000 non-null  object 
 1   tpep_dropoff_datetime  150000 non-null  object 
 2   passenger_count        126990 non-null  float64
 3   trip_distance          150000 non-null  float64
 4   PULocationID           150000 non-null  int64  
 5   DOLocationID           150000 non-null  int64  
 6   total_amount           150000 non-null  float64
dtypes: float64(3), int64(2), object(2)
memory usage: 8.0+ MB


# CONNECT KE DATA WAREHOUSE

In [6]:
conn = sqlite3.connect(DB_PATH)


# LOAD KE RAW TABLE (ELT)

In [7]:
df_taxi_raw.to_sql(
    "raw_taxi",
    conn,
    if_exists="replace",
    index=False
)


150000

# VERIFIKASI SQL

In [8]:
pd.read_sql(
    "SELECT name FROM sqlite_master WHERE type='table';",
    conn
)


,name
0,raw_taxi


# CEK ISI DATA

In [9]:
pd.read_sql(
    "SELECT * FROM raw_taxi LIMIT 5;",
    conn
)


,pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,PULocationID,DOLocationID,total_amount
0,2025-01-18 20:53:30,2025-01-18 21:00:47,1.0,0.97,238,166,13.32
1,2025-01-25 11:12:51,2025-01-25 11:17:57,1.0,0.60,50,48,10.55
2,2025-01-21 15:09:31,2025-01-21 15:19:02,1.0,0.80,236,237,16.65
3,2025-01-11 22:25:45,2025-01-11 22:34:22,2.0,1.93,231,68,20.37
4,2025-01-04 23:37:07,2025-01-04 23:45:58,NaN,4.44,137,88,35.12


# LOGGING

In [10]:
row_count = df_taxi_raw.shape[0]
col_count = df_taxi_raw.shape[1]

print("=== LOG ELT – NEW YORK TAXI RAW ===")
print("Sumber Data  : raw_taxi_data.csv")
print("Jumlah Baris :", row_count)
print("Jumlah Kolom :", col_count)
print("Waktu Load  :", datetime.now())


=== LOG ELT – NEW YORK TAXI RAW ===
Sumber Data  : .csv
Jumlah Baris : 150000
Jumlah Kolom : 7
Waktu Load  : 2026-01-07 18:56:45.143072
